<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> Montage is an astronomical image toolkit with components for reprojection, background matching, coaddition and visualization of FITS files. It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.

The Montage source is written in ANSI-C and code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ). The Python package can be installed from PyPI ("</i>pip install MontagePy<i>"). The package has no external dependencies. See http://montage.ipac.caltech.edu/ for details on the design and applications of Montage.


# MontagePy.main modules: mHistogram

The Montage image visualizer mViewer uses a histogram of the image values to determine the best stretch for display.  However, if we wish to use the same stretch for a set of images (<i>e.g.</i>, for tiled displays) we need to tie them all to the same histogram.  mHistogram uses the same code as mViewer to generate the histogram and color stretch bins and mViewer has an option to import this data instead of calculating it itself.

In [1]:
from MontagePy.main import mHistogram, mViewer

help(mHistogram)

Help on built-in function mHistogram in module MontagePy.main:

mHistogram(...)
    mHistogram generates a histogram from an image (to be used by mViewer mostly).
    
    Parameters
    ----------
    imgFile : str
        Input FITS file.
    histFile : str
        Output histogram file.
    minString : str
        Data range minimum as a string (to allow '%' and 's' suffix.
    maxString : str
        Data range maximum as a string (to allow '%' and 's' suffix.
    stretchType : str
        Stretch type (linear, power(log**N), sinh, gaussian or gaussian-log)
    grayLogPower : int, optional
        If the stretch type is log to a power, the power value.
    betaString : str, optional
        If the stretch type is asinh, the transition data value.
    debug : int, optional
        Debugging output level.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).
    minval : float
        Data value a

## mHistogram Example:

mViewer does not use the histogram directly but rather a set of stretch levels based on the histogram and the type of stretch the user wants.  There are several different stretches, from simple linear to a couple of gaussian-weighted histogram equalization techniques.  Even the starting and ending levels may be histogram-based and couched in terms of the distribution "sigma" values.

We will use the latter approach for this example:

In [2]:
rtn = mHistogram("M17/mosaic.fits", "work/M17/M17.hist", "-2s", "max", "gaussian-log")

print(rtn)

{'status': '0', 'minval': 149.5791825248529, 'minpercent': 0.0, 'minsigma': -2.0, 'maxval': 10639.0999189825, 'maxpercent': 100.0, 'maxsigma': 2240.4651757359084, 'datamin': 150.8148507701094, 'datamax': 10639.0999189825}


In [3]:
lncnt = 0
with open("work/M17/M17.hist", "r") as file:
    for line in file:  
        print(line, end='')
        lncnt = lncnt + 1
        if lncnt > 500:
            print('\n(Truncated at 500 lines)')
            break

# HISTOGRAM DATA
# 
# The following data is strictly formatted (after these initial comments).
# The first line is the type of stretch, i.e., power law (linear/log/etc.):0,
# gaussian:1, gaussian-log:2 or asinh:3.
# 
# The second are the data ranges the user gave in the various units (data value,
# percentiles, 'sigma' levels) plus the file data min, max, median and 'sigma'.
# 
# The third (a little repetitive) are the file statistics used in the histogram
# calculation: data min, max, the width of the bins ((max-min)/NBIN) and the 
# total number of pixels in the file.
# 
# Then the conclusions, starting with the 256 data values that correspond to 
# the lowest data value associated with a 'grayscale' output value.
# 
# Finally, the NBIN histogram values.  The first column is the bin number.
# The second is the lowest data value that will go into that bin.  This is
# the part that is stretch type dependent.  Next is the count of pixels that
# ended up in that bin followed by the cumul

&nbsp;<p/>

## Error Handling

If mHistogram encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies an image that doesn't exist: 

In [4]:
rtn = mHistogram("M17/unknown.fits", "work/M17/M17.hist", "-2s", "max", "gaussian-log")

print(rtn)

{'status': '1', 'msg': b'Image file M17/unknown.fits invalid FITS'}


&nbsp;

## Classic Montage: mHistogram as a Stand-Alone Program

### mHistogram Unix/Windows Command-line Arguments

<p>mHistogram can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mHistogram [-d] -file in.fits minrange maxrange [logpower/gaussian/gaussian-log/asinh [asinh-beta]] -out out.hist
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mHistogram can be accessed as a library function:</p>


<pre>
/*-*********************************************************************************************/
/*                                                                                             */
/*  mHistogram                                                                                 */
/*                                                                                             */
/*  This program is essentially a subset of mViewer containing the code that determines        */
/*  the stretch for an image.  With it we can generate stretch information for a whole         */
/*  file (or one reference file), then use it when we create a set of JPEG/PNG files           */
/*  from a set of other subsets/files.                                                         */
/*                                                                                             */
/*  Only one file is processed, so if we want color we need to run this program three          */
/*  times.  The only arguments needed are the image / stretch min / stretch max / mode.        */
/*                                                                                             */
/*   char *imgFile       Input FITS file.                                                      */
/*   char *histFile      Output histogram file.                                                */
/*   char *minString     Data range minimum as a string (to allow '%' and 's' suffix.          */
/*   char *maxString     Data range maximum as a string (to allow '%' and 's' suffix.          */
/*   char *stretchType   Stretch type (linear, power(log**N), sinh, gaussian or gaussian-log)  */
/*                                                                                             */
/*   int   grayLogPower  If the stretch type is log to a power, the power value.               */
/*   char *betaString    If the stretch type is asinh, the transition data value.              */
/*                                                                                             */
/*   int   debug         Debugging output level.                                               */
/*                                                                                             */
/***********************************************************************************************/

struct mHistogramReturn *mHistogram(char *grayfile, char *histfile, 
                                    char *grayminstr, char *graymaxstr, char *graytype, int graylogpower, char *graybetastr, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mHistogramReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   double minval;        // Data value associated with histogram minimum.
   double minpercent;    // Percentile value of histogram minimum.
   double minsigma;      // 'Sigma' level of histogram minimum.
   double maxval;        // Data value associated with histogram maximum.
   double maxpercent;    // Percentile value of histogram maximum.
   double maxsigma;      // 'Sigma' level of histogram maximum.
   double datamin;       // Minimum data value in file.
   double datamax;       // Maximum data value in file.
};
</pre>